In [128]:
import subprocess

intro_time = 60 
sim_duration = 300
trans_time = 90

n_init_rxtrs = 100
n_assem_core = 3
assem_size = 29565
init_total_assem_size = n_init_rxtrs * n_assem_core * assem_size
cycle_time = 8

In [129]:
init_uox_source = {'name' : 'SourceInitUOX', 
                   'config' : {'Source' : {'outcommod' : 'InitUOX',
                                           'outrecipe' : 'UOX_no232',
                                           'inventory_size' : init_total_assem_size
                                          }
                              }
                  }

In [105]:
from eg01_facilities import (natu_source, non_source, add_source, enrich, 
                             du_store, store_no232, mix_no232, store_232, 
                             mix_232)

In [130]:
lwr_full = {'name' : 'LWR',
            'lifetime' : 960,
            'config' : {'Reactor' : {'fuel_incommods' :  {'val' : ['UOX_Additive', 'UOX_NoAdditive', 'InitUOX']},
                                     'fuel_outcommods' : {'val' : ['SpentUOX_Additive', 'SpentUOX_NoAdditive', 'SpentUOX_NoAdditive']},
                                     'fuel_inrecipes' :  {'val' : ['UOX_232', 'UOX_no232', 'UOX_no232']},
                                     'fuel_outrecipes' : {'val' : ['SpentUOX_232', 'SpentUOX_no232', 'SpentUOX_no232']},
                                     'fuel_prefs' : {'val' : [1, 2, 2.5]},
                                     'pref_change_times' : {'val' : intro_time},
                                     'pref_change_commods' : {'val' : 'UOX_Additive'},
                                     'pref_change_values' : {'val' : 3},
                                     'cycle_time' : cycle_time,
                                     'refuel_time' : 0,
                                     'assem_size' : assem_size,
                                     'n_assem_core' : n_assem_core,
                                     'n_assem_batch' : 1,
                                     'power_cap' : 1000
                                    }
                       }
           }

In [107]:
eg23_sink = {'name' : 'Waste',
             'config' : {'Sink' : {'in_commods' : {'val' : 'Waste'}
                                  }
                        }
            }
# Post-LWR Updates
non_lwr_cool = {'name' : 'CoolingUOX_NoAdditive',
                'config' : {'Storage' : {'in_commods' : {'val' : 'SpentUOX_NoAdditive'},
                                         'out_commods' : {'val' : 'CooledSpentUOX_NoAdditive'},
                                         'in_recipe' : 'SpentUOX_no232',
                                         'residence_time' : 10#81
                                        }
                           }
               }
add_lwr_cool = {'name' : 'CoolingUOX_Additive',
                'config' : {'Storage' : {'in_commods' : {'val' : 'SpentUOX_Additive'},
                                         'out_commods' : {'val' : 'CooledSpentUOX_Additive'},
                                         'in_recipe' : 'SpentUOX_232',
                                         'residence_time' : 10#81
                                        }
                           }
               }
sep_streams = {'item' : [{'commod' : 'LWR_Pu',
                          'info' : {'buf_size' : 1e100,
                                    'efficiencies' : {'item' : {'comp': 'Pu', 'eff': 0.99}}}},
                         {'commod' : 'LWR_RU', 
                          'info' : {'buf_size' : 1e100, 
                                    'efficiencies' : {'item' : {'comp': 'U', 'eff': 0.99}}}}
                        ]
              }
lwr_sep = {'name' : 'SepLWR',
           'config' : {'Separations' : {'leftover_commod' : 'Waste',
                                        'throughput' : 1e100,
                                        'feedbuf_size' : 1e100,
                                        'feed_commods' : {'val' : ['CooledSpentUOX_NoAdditive', 'CooledSpentUOX_Additive']},
                                        'feed_commod_prefs' : {'val' : [1, 2]},
                                        'streams' : sep_streams}}
      }

In [131]:
init_lwr_prototypes = ['LWR' for x in range(0, n_init_rxtrs)]
n_builds = [1 for x in range(0, n_init_rxtrs)]
build_times = [x + 1 for x in range(0, 17) for y in range(0,6)]
del build_times[-3:-1]
# Lifetimes borrowed from previous EG scenario work presuming a 
# start year of 2000 and first decommission in 2015. Updated to 
# better stagger decommissioning.
old_lives = [181, 186, 191, 196, 
             201, 206, 211, 216, 221, 226, 231, 236, 241, 246, 251, 256, 261, 266, 271, 276, 281, 286, 291, 296,
             301, 306, 311, 316, 321, 326, 331, 336, 341, 346, 351, 356, 361, 366, 371, 376, 381, 386, 391, 396,
             401, 406, 411, 416, 421, 426, 431, 436, 441, 446, 451, 456, 461, 466, 471, 476, 481, 486, 491, 496,
             501, 506, 511, 516, 521, 526, 531, 536, 541, 546, 551, 556, 561, 566, 571, 576, 581, 586, 591, 596,
             601, 606, 611, 616, 621, 626, 631, 636, 641, 646, 651, 656, 661, 666, 671, 676
            ]
# Overwrite lifetimes to have decommission start 1.5 year in, 
# expecting a sim start of 2022
lifetimes = [x - 163 for x in old_lives]

In [132]:
init_fleet = {'name' : 'InitFleet', 
              'config' : {'DeployInst' : {'prototypes' :  {'val' : init_lwr_prototypes},
                                          'n_build' :     {'val' : n_builds},
                                          'build_times' : {'val' : build_times},
                                          'lifetimes' :   {'val' : lifetimes}
                                         }
                         }
             }

In [118]:
lin_func = {'piece' : [{'start' : 18,
                        'function' : {'type' : 'linear',
                                      'params' : '0 100000'
                                     }
                       }]
           }

growth_region = {'GrowthRegion' : {'growth' : {'item' : [{'commod' : 'power',
                                                          'piecewise_function' : lin_func
                                                         }]
                                              }
                                  }
                }

In [119]:
dep_u = {'name' : 'DU',
         'basis' : 'mass',
         'nuclide' : [{'id' : 'U235', 'comp' : 0.0025}, 
                      {'id' : 'U238', 'comp' : 0.9975}]
        }
nat_u = {'name' : 'NU',
         'basis' : 'mass',
         'nuclide' : [{'id' : 'U235', 'comp' : 0.007110}, 
                      {'id' : 'U238', 'comp' : 0.992890}]
        }
from recipe_100ppt import (isos_no232, isos_232, enr_no232, enr_232, 
                           uox_no232, uox_10pct232, uox_50pct232, uox_232, 
                           spent_no232, spent_10pct232, spent_50pct232, 
                           spent_232, ff_no232, ff_232, spentff_no232, 
                           spentff_232
                          )

In [120]:
s1 = {'info' : {'mixing_ratio' : 0.5,
                'buf_size' : 1e7}, 
                'commodities' : {'item' : {'commodity' : 'LWR_Pu', 'pref' : 1}}
     }
s2 = {'info' : {'mixing_ratio' : 0.5,
                'buf_size' : 1e7}, 
                'commodities' : {'item' : {'commodity' : 'LWR_RU', 'pref' : 1}}
     }
mix = {'name' : 'Mix',
       'config' : {'Mixer' : {'in_streams' : {'stream' : [s1, s2]},
                              'out_commod' : 'FakeFuel',
                              'throughput' : 1e10,
                              'out_buf_size' : 1e20}}
      }

In [121]:
eg1_23_fc_prototypes = ['SourceNatU', 'SourceNoAdditiveIsos', 'SourceAdditiveIsos', 'Enrichment', 'StorageDepU', 
                        'StorageNoAdditive', 'MixerNoAdditive', 'StorageAdditive', 'MixerAdditive', 
                        'LWR', 'CoolingUOX_NoAdditive', 'CoolingUOX_Additive', 'Waste'
                       ]

eg1_23_fc_inst = {'name' : 'FCInstEG01', 
                  'initialfacilitylist' : {'entry' : [{'number' : 1, 'prototype' : 'SourceInitUOX'},
                                                      {'number' : 1, 'prototype' : 'SourceNatU'},
                                                      {'number' : 1, 'prototype' : 'SourceNoAdditiveIsos'},
                                                      {'number' : 1, 'prototype' : 'SourceAdditiveIsos'},
                                                      {'number' : 1, 'prototype' : 'Enrichment'},
                                                      {'number' : 1, 'prototype' : 'StorageDepU'},
                                                      {'number' : 1, 'prototype' : 'StorageNoAdditive'},
                                                      {'number' : 1, 'prototype' : 'MixerNoAdditive'},
                                                      {'number' : 1, 'prototype' : 'StorageAdditive'},
                                                      {'number' : 1, 'prototype' : 'MixerAdditive'},
                                                      {'number' : 1, 'prototype' : 'CoolingUOX_NoAdditive'},
                                                      {'number' : 1, 'prototype' : 'CoolingUOX_Additive'},
                                                      {'number' : 1, 'prototype' : 'SepLWR'}, ###########
                                                      {'number' : 1, 'prototype' : 'Mix'}, ###########
                                                      {'number' : 1, 'prototype' : 'Waste'}]
                                          },
                  'config' : {'ManagerInst' : {'prototypes' : {'val' : eg1_23_fc_prototypes}
                                              }
                             }
                 }

In [126]:
control = {'duration' : sim_duration, 
           'startmonth' : 1, 
           'startyear' : 2022,
           'dt' : 86400
          }
archetypes = {'spec' : [{'lib' : 'cycamore', 'name' : 'Source'},
                        {'lib' : 'cycamore', 'name' : 'Enrichment'},
                        {'lib' : 'cycamore', 'name' : 'Mixer'},
                        {'lib' : 'cycamore', 'name' : 'Reactor'},
                        {'lib' : 'cycamore', 'name' : 'Storage'},
                        {'lib' : 'cycamore', 'name' : 'Separations'},
                        {'lib' : 'cycamore', 'name' : 'Sink'},
                        {'lib' : 'cycamore', 'name' : 'DeployInst'},
                        {'lib' : 'cycamore', 'name' : 'ManagerInst'},
                        {'lib' : 'cycamore', 'name' : 'GrowthRegion'},
                      ]
             }
# full additive availability at intro
full_region = {'name' : 'GrowthRegion', 
               'config' : growth_region, 
               'institution' : [init_fleet, eg1_23_fc_inst]
              }
full_sim = {'simulation' : {'control' : control,
                            'archetypes' : archetypes,
                            'region' : full_region,
                            'facility' : [init_uox_source, natu_source, non_source, add_source, enrich,
                                          du_store, store_no232, mix_no232, store_232, mix_232, lwr_full, 
                                          non_lwr_cool, add_lwr_cool, mix, lwr_sep,
                                          eg23_sink
                                         ],
                            'recipe' : [dep_u, nat_u, isos_no232, isos_232, enr_no232, enr_232, 
                                        uox_no232, uox_232, spent_no232, spent_232
                                       ]
                           }
           }

In [127]:
filebase = 'test_py'

in_file = filebase + '.py'
sim_file = '../output/' + filebase + '.sqlite'

with open(in_file, 'w') as file: 
    file.write('SIMULATION = ' + str(full_sim))
subprocess.run(['rm', sim_file])
subprocess.run(['cyclus', in_file, '-o', sim_file])

CompletedProcess(args=['cyclus', 'test_py.py', '-o', '../output/test_py.sqlite'], returncode=-6)